In [14]:
# coding=utf-8
import numpy as np
import pandas as pd
import math
import nltk

In [15]:
d = pd.DataFrame()
# cities = ["nyc", "boston", "portland", "sf", "oakland", "la", "seattle", "chicago", "dc", 
#           "montreal", "toronto" ,"sydney", "london", "melbourne"]
cities = ["boston"]
for c in cities:
    tmp_d = pd.read_csv('listings_' + c + '.csv',sep=',',dtype=pd.np.str)
    tmp_d['listing_city'] = c
    d = d.append(tmp_d, ignore_index=True)

In [16]:
# coding=utf-8
desc_sent = []
for s in d["description"]:
    s=str(s).strip().decode("ascii","ignore").encode("ascii")
    desc_sent.append(nltk.sent_tokenize(s))

In [17]:
self_reference_dict = ["I", "me", "my", "we", "us", "our", "mine", "myself", "ourselves", "ours"]

In [18]:
self_reference_score = []
for sents in desc_sent:
    cnt = 0
    flg = False
    if (len(sents) == 0): 
        self_reference_score.append(0)
        continue
    for s in sents:
        words = nltk.word_tokenize(s)
        for w in self_reference_dict:
            if w in words:
                cnt += 1
                flg = True
                break
        if (flg):
#             print s
            flg = False
    self_reference_score.append(cnt/float(len(sents)))

In [19]:
d = d.join(pd.DataFrame({'self_reference_score': self_reference_score}))

In [20]:
tot_self_reference_score = pd.DataFrame(data=np.zeros((30, len(cities))), columns=cities)
cnt = pd.DataFrame(data=np.zeros((30, len(cities))), columns=cities)
ave_self_reference_score = pd.DataFrame(data=np.zeros((30, len(cities))), columns=cities)

for i,score in enumerate(d['number_of_reviews']):
    tot_self_reference_score[d['listing_city'][i]][min(int(score)/2, 29)] += d['self_reference_score'][i]
    cnt[d['listing_city'][i]][min(int(score)/2, 29)] += 1

        # print "review_scores_value\tlistings_number"
# for i in range(10):
#     print i+1, "\t\t\t", int(cnt[i])
print "\n"
print "number_of_reviews\taverage_self_reference_score"
for c in cities:
    print c
    for i in range(30):
        ave_self_reference_score[c][i] = 0
        if float(cnt[c][i]) > 0:
            ave_self_reference_score[c][i] = float(tot_self_reference_score[c][i])/int(cnt[c][i])
            print i*2, "\t\t\t", ave_self_reference_score[c][i]



number_of_reviews	average_self_reference_score
boston
0 			0.0873680237395
2 			0.109071899768
4 			0.122353887895
6 			0.134327777109
8 			0.107810697583
10 			0.120079098637
12 			0.150360202325
14 			0.11699735713
16 			0.0765649672689
18 			0.144762653022
20 			0.0780506258447
22 			0.111453831605
24 			0.159150621588
26 			0.115010158192
28 			0.168626036273
30 			0.125132247926
32 			0.118647223472
34 			0.125065581082
36 			0.229513872371
38 			0.21808683625
40 			0.222889957265
42 			0.124615420811
44 			0.133634809635
46 			0.187332962875
48 			0.086191749427
50 			0.0420512820513
52 			0.191744366744
54 			0.238492063492
56 			0.14855977356
58 			0.121856843381


In [21]:
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
plt.plot(d['number_of_reviews'], d['average_reference_score'], 'r.')

KeyError: 'average_reference_score'

In [ ]:
from math import pi

from bokeh.models import HoverTool
from bokeh.plotting import ColumnDataSource, figure, show, output_file
from bokeh.io import output_notebook

In [ ]:
scores = range(0,60,2)

colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce",
          "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]

city = []
score = []
color = []
rate = []
for c in cities:
    for s in scores:
        city.append(c)
        score.append(s/2)
        rate.append(ave_self_reference_score[c][s/2])
        color.append(colors[min(max(int((ave_self_reference_score[c][s/2]*2-0.15)*80/3), 0), 8)])
        s += 1

source = ColumnDataSource(
    data=dict(city=city, score=score, color=color, rate=rate)
)

TOOLS = "hover"

p = figure(title="AirBnB-self-reference",
           x_range=map(str, scores), y_range=list(reversed(cities)),
           x_axis_location="below", plot_width=500, plot_height=500,
           toolbar_location="left", tools=TOOLS)

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "5pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = pi/3

p.rect("score", "city", width=1, height=1, source=source,
       color="color", line_color=None)

p.select_one(HoverTool).tooltips = [
    ('city : review number', '@city : @score'),
    ('self-reference-rate', '@rate'),
]

output_notebook()

show(p)